In [ ]:
#cm1

# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score
import xgboost as xgb
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib
import warnings
warnings.filterwarnings('ignore')

# Function to read mass spectra data from CSV
def load_spectrum(filepath):
    """Reads CSV file containing mass spectrum data
    Returns: m/z values and intensity values"""
    try:
        spectrum = pd.read_csv(filepath, skiprows=1)  # Skip header row
        mz = spectrum['X(Thompsons)'].values         # Mass/charge values
        intensities = spectrum['Y(Counts)'].values    # Intensity values
        return mz, intensities
    except Exception as e:
        if 'skiprows' not in str(e):
            print(f"Error loading {filepath}: {str(e)}")
        return None, None

# Function to preprocess and align two spectra
def align_spectra(mz1, intensities1, mz2, intensities2, n_points=1000):
    """Aligns two spectra for comparison by:
    1. Creating common m/z axis
    2. Interpolating intensities
    3. Normalizing intensities to [0,1] range"""
    try:
        # Find overlapping m/z range
        min_mz = max(np.min(mz1), np.min(mz2))
        max_mz = min(np.max(mz1), np.max(mz2))
        
        # Create uniform m/z axis with 1000 points
        common_mz = np.linspace(min_mz, max_mz, n_points)
        
        # Interpolate intensities to match common m/z axis
        f1 = interp1d(mz1, intensities1, kind='linear', bounds_error=False, fill_value=0)
        f2 = interp1d(mz2, intensities2, kind='linear', bounds_error=False, fill_value=0)
        
        int1_aligned = f1(common_mz)
        int2_aligned = f2(common_mz)
        
        # Normalize intensities to [0,1] range
        max_int1 = np.max(int1_aligned)
        max_int2 = np.max(int2_aligned)
        
        if max_int1 > 0:
            int1_aligned = int1_aligned / max_int1
        if max_int2 > 0:
            int2_aligned = int2_aligned / max_int2
        
        return common_mz, int1_aligned, int2_aligned
        
    except Exception as e:
        print(f"Error aligning spectra: {str(e)}")
        return None, None, None

# Function to calculate similarity features between two spectra
def calculate_similarity_features(mz, int1, int2):
    """Calculates 3 similarity metrics between spectra:
    1. Cosine similarity: angle between spectra vectors
    2. Area ratio: ratio of areas under curves
    3. Standard deviation ratio: ratio of intensity spreads"""
    try:
        # Calculate cosine similarity (angle between vectors)
        cosine_sim = np.dot(int1, int2) / (np.linalg.norm(int1) * np.linalg.norm(int2))
        
        # Calculate area ratios using trapezoidal integration
        area1 = np.trapz(int1, mz)
        area2 = np.trapz(int2, mz)
        area_ratio = min(area1/area2, area2/area1) if area1 > 0 and area2 > 0 else 0
        
        # Calculate standard deviation ratios
        std_ratio = min(np.std(int1)/np.std(int2), np.std(int2)/np.std(int1)) if np.std(int1) > 0 and np.std(int2) > 0 else 0
        
        features = {
            'cosine_similarity': cosine_sim,
            'area_ratio': area_ratio,
            'std_ratio': std_ratio
        }
        
        # Check for invalid values
        if not all(np.isfinite(v) for v in features.values()):
            return None
            
        return features
        
    except Exception as e:
        print(f"Error calculating features: {str(e)}")
        return None

# Function to check if compound files exist
def check_compound_existence(base_path, compound_num):
    """Checks if at least one spectrum file exists for a compound"""
    for i in range(1, 11):
        if os.path.exists(f"{base_path}/{compound_num:02d}-{i:02d}.csv"):
            return True
    return False

# Main function to generate comparison dataset
def generate_balanced_comparisons(base_path, debug=False):
    """Creates balanced dataset of spectrum comparisons:
    - Positive class (y=1): Same compound comparisons
    - Negative class (y=0): Different compound comparisons"""
    X = []  # Features
    y = []  # Labels
    comparisons = []  # File pairs
    same_count = 0    # Count of same-compound pairs
    diff_count = 0    # Count of different-compound pairs
    
    # Find all existing compounds
    existing_compounds = []
    for compound_i in range(1, 29):  # CM1 has 28 compounds
        if check_compound_existence(base_path, compound_i):
            existing_compounds.append(compound_i)
    
    if debug:
        print(f"\nFound {len(existing_compounds)} compounds")
        
    # Generate same-compound comparisons
    for compound_i in existing_compounds:
        if debug:
            print(f"Processing same-compound comparisons for compound {compound_i}")
            
        # Compare first 5 spectra of each compound
        for i in range(1, 5):
            for j in range(i + 1, 6):
                file1 = f"{base_path}/{compound_i:02d}-{i:02d}.csv"
                file2 = f"{base_path}/{compound_i:02d}-{j:02d}.csv"
                
                if not os.path.exists(file1) or not os.path.exists(file2):
                    continue
                    
                features = calculate_similarity_features(
                    *align_spectra(*load_spectrum(file1), *load_spectrum(file2))
                )
                
                if features is not None:
                    X.append(list(features.values()))
                    y.append(1)  # Same compound
                    comparisons.append((file1, file2))
                    same_count += 1
    
    # Generate different-compound comparisons
    for idx, compound_i in enumerate(existing_compounds[:-1]):
        if debug:
            print(f"Processing different-compound comparisons for compound {compound_i}")
            
        # Compare with next 2 compounds for balance
        for compound_j in existing_compounds[idx+1:min(idx+3, len(existing_compounds))]:
            for i in range(1, 6):
                for j in range(1, 6):
                    file1 = f"{base_path}/{compound_i:02d}-{i:02d}.csv"
                    file2 = f"{base_path}/{compound_j:02d}-{j:02d}.csv"
                    
                    if not os.path.exists(file1) or not os.path.exists(file2):
                        continue
                    
                    features = calculate_similarity_features(
                        *align_spectra(*load_spectrum(file1), *load_spectrum(file2))
                    )
                    
                    if features is not None:
                        X.append(list(features.values()))
                        y.append(0)  # Different compounds
                        comparisons.append((file1, file2))
                        diff_count += 1
    
    # Print dataset statistics
    print("\nComparison Statistics:")
    print(f"Same compound comparisons: {same_count}")
    print(f"Different compound comparisons: {diff_count}")
    print(f"Class balance ratio (same/diff): {same_count/diff_count:.2f}")
    
    if len(X) == 0:
        print("No valid comparisons generated!")
        return None, None, None
        
    return np.array(X), np.array(y), comparisons

# Function to visualize confusion matrix
def plot_confusion_matrix(y_true, y_pred):
    """Creates and plots confusion matrix for model evaluation"""
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Different', 'Same'],
                yticklabels=['Different', 'Same'])
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()
    return cm

# Main execution function
def main():
    # Set data path
    base_path = '/Users/ajibolaoluwatobiloba/Desktop/Mass spectra project/minmax_SourceAndData/C-ComputingIndices/CosineSimilarity/CM1'
    
    # Generate dataset
    print("Generating CM1 comparisons with tuned parameters...")
    X, y, comparisons = generate_balanced_comparisons(base_path, debug=True)
    
    if X is not None:
        # Split data into train/test sets and scale features
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
        
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # Create XGBoost model with optimized parameters
        model = xgb.XGBClassifier(
            # Tree parameters
            max_depth=4,
            min_child_weight=2,
            gamma=0.1,
            
            # Sampling parameters
            subsample=0.9,
            colsample_bytree=0.9,
            
            # Learning parameters
            learning_rate=0.005,
            n_estimators=300,
            
            # Class weight for imbalance
            scale_pos_weight=4,
            
            # Regularization
            reg_lambda=0.8,
            reg_alpha=0.1,
            
            # Other settings
            objective='binary:logistic',
            random_state=42,
            use_label_encoder=False
        )
        
        # Perform cross-validation
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=cv, scoring='accuracy')
        
        print("\nCross-validation scores:", cv_scores)
        print(f"Mean CV score: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
        
        # Train final model
        print("\nTraining model...")
        model.fit(X_train_scaled, y_train)
        
        # Evaluate model performance
        y_pred = model.predict(X_test_scaled)
        y_prob = model.predict_proba(X_test_scaled)[:, 1]
        
        print("\nTest Set Performance:")
        print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
        print(f"ROC AUC: {roc_auc_score(y_test, y_prob):.4f}")
        print("\nClassification Report:")
        print(classification_report(y_test, y_pred))
        
        # Analyze feature importance
        feature_names = ['Cosine Similarity', 'Std Ratio', 'Area Ratio']
        importance = model.feature_importances_
        
        # Plot feature importance
        plt.figure(figsize=(10, 6))
        importance_df = pd.DataFrame({
            'Feature': feature_names,
            'Importance': importance
        })
        importance_df = importance_df.sort_values('Importance', ascending=True)
        
        sns.barplot(x='Importance', y='Feature', data=importance_df, palette='viridis')
        plt.title('Feature Importance in CM1 Spectrum Comparison')
        plt.xlabel('Importance Score')
        plt.tight_layout()
        plt.show()
        
        # Print feature rankings
        print("\nFeature Importance Rankings:")
        for name, imp in sorted(zip(feature_names, importance), key=lambda x: x[1], reverse=True):
            print(f"{name:20} {imp:.4f}")
        
        # Show confusion matrix
        print("\nConfusion Matrix:")
        cm = plot_confusion_matrix(y_test, y_pred)
        
       

# Execute main function if script is run directly
if _name_ == "_main_":
    main()